In [1]:
# !pip install sqlalchemy
# !pip install pymysql

In [2]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
import sys
sys.path.append('..')
from config import remote_gwsis_dbuser, remote_gwsis_dbpwd, remote_db_host, remote_db_port, remote_gwsis_dbname

# Import Pandas
import pandas as pd

In [3]:
# Cloud MySQL Database Connection on AWS
engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_host}:{remote_db_port}/{remote_gwsis_dbname}")

In [4]:
# Create a remote database engine connection
conn = engine.connect()

In [5]:
state_codes = pd.read_csv('geojson/state_code.csv')
state_codes.head()

,State/District,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [6]:
condition_data = pd.read_csv('tunnel/Tunnel_Condition_Summary.csv')
condition_data.head()

,State,Total,Condition_State_1,Condition_State_2,Condition_State_3,Condition_State_4,Condition_State_1_Percentage,Condition_State_2_Percentage,Condition_State_3_Percentage,Condition_State_4_Percentage
0,Alabama,1998282,1725296,251583,21191,212,0.863390,0.125900,0.010605,0.000106
1,Alaska,1098774,353522,511542,188371,45339,0.321742,0.465557,0.171437,0.041263
2,Arizona,1750631,1554508,164065,28904,3154,0.887970,0.093718,0.016511,0.001802
3,Arkansas,376491,366899,9322,269,1,0.974523,0.024760,0.000714,0.000003
4,California,16724871,15872091,309136,321655,221989,0.949011,0.018484,0.019232,0.013273


In [7]:
tunnel_summary = condition_data.merge(state_codes, left_on=["State"], right_on=["State/District"])
tunnel_summary = tunnel_summary[["State",
                                 "Total",
                                 "Condition_State_1",
                                 "Condition_State_2",
                                 "Condition_State_3",
                                 "Condition_State_4",
                                 "Condition_State_1_Percentage",
                                 "Condition_State_2_Percentage",
                                 "Condition_State_3_Percentage",
                                 "Condition_State_4_Percentage",
                                 "Postal Code"
                            ]]
tunnel_summary = tunnel_summary.rename(columns={"Postal Code":"State_Abbreviation"})
tunnel_summary.head()

,State,Total,Condition_State_1,Condition_State_2,Condition_State_3,Condition_State_4,Condition_State_1_Percentage,Condition_State_2_Percentage,Condition_State_3_Percentage,Condition_State_4_Percentage,State_Abbreviation
0,Alabama,1998282,1725296,251583,21191,212,0.863390,0.125900,0.010605,0.000106,AL
1,Alaska,1098774,353522,511542,188371,45339,0.321742,0.465557,0.171437,0.041263,AK
2,Arizona,1750631,1554508,164065,28904,3154,0.887970,0.093718,0.016511,0.001802,AZ
3,Arkansas,376491,366899,9322,269,1,0.974523,0.024760,0.000714,0.000003,AR
4,California,16724871,15872091,309136,321655,221989,0.949011,0.018484,0.019232,0.013273,CA


In [8]:
tunnel_summary.to_sql(name='tunnel_condition_summary', if_exists='replace', con=conn, chunksize=500, index=False)